In [1]:
!pip install nba_api

In [3]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog, commonplayerinfo
import pandas as pd

In [21]:
def get_player_id(player_name):
    for p in players.get_players():
        if p['full_name'].lower() == player_name.lower():
            return p['id']

    return None

def get_player_career_start_year(player_id):
    player_info = commonplayerinfo.CommonPlayerInfo(player_id = player_id)
    career_start_year = player_info.get_data_frames()[0]['FROM_YEAR'][0]

    return int(career_start_year)

player_id = get_player_id('stephen curry')
print(get_player_career_start_year(player_id))

2009
